#### Making pickle files

Notebook which produces data which is then plotted in plotting_script.ipynb.

In [ ]:
import numpy as np
import subprocess
import pickle as pkl

#### Functions

In [2]:
def pickle_save_and_open(file, string_file):        
    print("dumping")
    with open(f'{string_file}.pkl', 'wb') as handle:
        pkl.dump(file, handle, protocol=pkl.HIGHEST_PROTOCOL)
        
    print("loading")
    with open(f'{string_file}.pkl', 'rb') as handle:
        file = pkl.load(handle)
    return file

In [3]:
def run_subprocess(script, steps=100, size=50, temp=0.5, plotflag=0, num_threads=None):
    if num_threads is not None:
        output = subprocess.run(["python", script, f"{steps}", f"{size}", f"{temp}", f"{plotflag}", f"{num_threads}"], capture_output=True)
    else:
        output = subprocess.run(["python", script, f"{steps}", f"{size}", f"{temp}", f"{plotflag}"], capture_output=True)
    return str(output)
        

#### Compile cython scripts

In [ ]:
setup_message = str( subprocess.run(["python", "setup_cython.py", "build_ext", "-fi"], capture_output=True) )
parallel_setup_message = str( subprocess.run(["python", "setup_parallel_cython.py", "build_ext", "-fi"], capture_output=True) )
print(setup_message)
print(parallel_setup_message)

In [ ]:
all_scripts = ["LebwohlLasher.py", "NumpyVecLebwohlLasher.py", "NumbaLebwohlLasher.py", "run_cython.py", "run_parallel_cython.py", "mpi4pyLebwohlLasher.py", "run_mpi_cython.py"]
slow_scripts = ["mpi4pyLebwohlLasher.py", "LebwohlLasher.py"]#, "run_mpi_cython.py"]
mpi_scripts = ["mpi4pyLebwohlLasher.py", "run_mpi_cython.py"]
fast_scripts = ["NumpyVecLebwohlLasher.py", "NumbaLebwohlLasher.py", "run_cython2.py", "run_parallel_cython.py"]


In [ ]:
all_lattice_sizes = [25, 50, 100, 150, 200]
fast_lattice_sizes = [25, 50, 100, 150, 300, 450, 700]
numba_threads_lattice_sizes =[25,50,100,150,200,400,700]


In [ ]:
# all scripts
repeats = 7

try:
    with open('all_scripts.pkl', 'rb') as handle:
        all_scripts = pkl.load(handle)

except:
    all_scripts = {}
    for script in all_scripts:
        print(f"RUNNING {script}")
        times = np.zeros(len(all_lattice_sizes))
        stds = np.zeros(len(all_lattice_sizes))
        for i, size in enumerate(all_lattice_sizes):
            print(f"lattice side length: {size}")
            if size > 175 and (script == "LebwohlLasher.py" or script == "mpi4pyLebwohlLasher.py"):
                repeats = 1
            elif size > 125 and (script == "LebwohlLasher.py" or script == "mpi4pyLebwohlLasher.py"):
                repeats = 3
            else: 
                repeats = 4
            
            time_value_reps = np.zeros(repeats)
            
            if size > 275 and script == "LebwohlLasher.py":
                av_time_value = None
                std_time_value = None
            elif size > 125 and script == "run_mpi_cython.py":
                av_time_value = None
                std_time_value = None
            else:  
                for j in range(repeats):
                
                    if script == "mpi4pyLebwohlLasher.py" or script =="run_mpi_cython.py":
                        num_threads = 4
                        output = str( subprocess.run(["mpiexec", "-n", f"{num_threads}", "python", script, "100", f"{size}", "0.5", "0"], capture_output=True) )

                    elif script == "run_parallel_cython.py":
                        num_threads = 2
                        output = run_subprocess(script, size=size, num_threads=num_threads)
                        
                    elif script == "NumbaLebwohlLasher.py":
                        num_threads = 4
                        output = run_subprocess(script, size=size, num_threads=num_threads)
                        
                    else:
                        output = run_subprocess(script, size=size)
                        
                    time_value = float(output.split("Time: ")[1][:7])
                

                    time_value_reps[j] = float(output.split("Time: ")[1][:7])
                        # print(f"time{i}: {time_value_rep}")
                        
                av_time_value = np.mean(time_value_reps)
                std_time_value = np.std(time_value_reps)
            
            times[i] = (av_time_value)
            stds[i] = (std_time_value)
        print(f"times: {times}")
        print(f"stds: {stds}")
        all_scripts[f"{script}_times"] = times
        all_scripts[f"{script}_stds"] = stds
    
    all_scripts = pickle_save_and_open(all_scripts, "all_scripts")

In [ ]:
#  mpi comparison
repeats = 5
thread = 4

try:
    with open('all_scripts.pkl', 'rb') as handle:
        all_scripts = pkl.load(handle)

except:


    mpi_comp = {}
    for script in mpi_scripts:
            times = np.zeros(len(all_lattice_sizes)) 
            stds = np.zeros(len(all_lattice_sizes))
            for i, size in enumerate(all_lattice_sizes):
                # print(f"{thread} threads")
                time_value_tot = 0
                time_value_reps = np.zeros(repeats)
                for j in range(repeats):
                    output = str( subprocess.run(["mpiexec", "-n", f"{num_threads}", "python", script, "100", f"{size}", "0.5", "0"], capture_output=True) )
                    
                    # print(float(output.split("Time: ")[1][:7]))
                    time_value_reps[j] = float(output.split("Time: ")[1][:7])
                    # print(f"time{i}: {time_value_rep}")
                    
                av_time_value = np.mean(time_value_reps)
                std_time_value = np.std(time_value_reps)
                
                times[i] = (av_time_value)
                stds[i] = (std_time_value)
            print(f"times: {times}")
            print(f"stds: {stds}")
            mpi_comp[f"{script}_times"] = times
            mpi_comp[f"{script}_stds"] = stds
            # print(mpi_comp)
            
    mpi_comp = pickle_save_and_open(mpi_comp, "mpi_comp")

In [ ]:
# numba multi threads

# subprocess.run("python setup_parallel_cython.py build_ext -fi")

num_threads = [1,2,4]
repeats = 4

try:
    with open('parallel_numba_threads.pkl', 'rb') as handle:
        parallel_numba_threads = pkl.load(handle)
    print("parallel_numba_threads loaded in")
    
except:
    parallel_numba_threads = {}
    script = "NumbaLebwohlLasher.py"
    print(f"RUNNING {script}")
    for thread in (num_threads):
        times = np.zeros(len(numba_threads_lattice_sizes))
        stds = np.zeros(len(numba_threads_lattice_sizes))
        print(f"threads: {thread}")
        for i, size in enumerate(numba_threads_lattice_sizes):
            # print(f"{thread} threads")
            time_value_tot = 0
            time_value_reps = np.zeros(repeats)
            for j in range(repeats):
                output = run_subprocess(script, size=size, num_threads=thread)
                
                
                # print(float(output.split("Time: ")[1][:7]))
                time_value_reps[j] = float(output.split("Time: ")[1][:7])
                # print(f"time{i}: {time_value_rep}")
                
            av_time_value = np.mean(time_value_reps)
            std_time_value = np.std(time_value_reps)
            
            times[i] = (av_time_value)
            stds[i] = (std_time_value)
        print(f"times: {times}")
        print(f"stds: {stds}")
        parallel_numba_threads[f"{script}_thread{thread}_times"] = times
        parallel_numba_threads[f"{script}_thread{thread}_stds"] = stds
        # print(parallel_numba_threads)

    parallel_numba_threads = pickle_save_and_open(parallel_numba_threads, "parallel_numba_threads")




In [ ]:
# fast scripts 

repeats = 5

try:
    with open('fast_scripts_lattice_size.pkl', 'rb') as handle:
        fast_scripts_lattice_size = pkl.load(handle)

    print("fast_scripts_lattice loaded in")
    
except:
    fast_scripts_lattice_size = {}
    for script in fast_scripts:
        print(f"RUNNING {script}")
        times = np.zeros(len(fast_lattice_sizes))
        stds = np.zeros(len(fast_lattice_sizes))
        for i, size in enumerate(fast_lattice_sizes):
            print(f"lattice side length: {size}")
            if size > 200:
                repeats = 2
            else:
                repeats = 5
            time_value_tot = 0
            time_value_reps = np.zeros(repeats)
            for j in range(repeats):
                if script == "run_parallel_cython.py" :
                    output = run_subprocess(script, size=size, num_threads=2)
                elif script == "NumbaLebwohlLasher.py":
                    output = run_subprocess(script, size=size, num_threads=4)
                else:
                    output = run_subprocess(script, size=size)
                
                time_value_reps[j] = float(output.split("Time: ")[1][:7])
                
            av_time_value = np.mean(time_value_reps)
            std_time_value = np.std(time_value_reps)
            
            times[i] = (av_time_value)
            stds[i] = (std_time_value)
        print(f"times: {times}")
        print(f"stds: {stds}")
        fast_scripts_lattice_size[f"{script}_times"] = times
        fast_scripts_lattice_size[f"{script}_stds"] = stds
        
    fast_scripts_lattice_size = pickle_save_and_open(fast_scripts_lattice_size, "fast_scripts_lattice_size")


RUNNING run_cython2.py
lattice side length: 25
lattice side length: 50
lattice side length: 100
lattice side length: 150
lattice side length: 300
lattice side length: 450
lattice side length: 700
times: [ 0.02849   0.102706  0.343622  0.757448  3.11654   6.75385  16.326   ]
stds: [0.00357495 0.02238015 0.00384453 0.01820798 0.05139    0.03402
 0.1957    ]
dumping
loading


In [ ]:
# orde rvs temp

temps = np.arange(0.1, 1.7, 0.05)
repeats = 10
nmax=25


try:

    with open('temp_order.pkl', 'rb') as handle:
        temp_order = pkl.load(handle)

    print("temp_order loaded in")
    
except:
    temp_order = {}
    script = "run_cython.py"
    print(f"RUNNING {script}")
    orders = np.zeros(len(temps))
    stds = np.zeros(len(temps))
    for i, temp in enumerate(temps):
        print(f"{temp} degrees?")
        order_value_tot = 0
        order_value_reps = np.zeros(repeats)
        for j in range(repeats):
            output = run_subprocess(script, steps=1000, size=nmax, temp=temp)
            order_value_reps[j] = (float(output.split("OrderAv: ")[1][:5]))
            
        av_time_value = np.mean(order_value_reps)
        std_time_value = np.std(order_value_reps)
        
        orders[i] = (av_time_value)
        stds[i] = (std_time_value)
        
    print(f"orders: {orders}")
    print(f"stds: {stds}")
    temp_order[f"{script}_orders"] = orders
    temp_order[f"{script}_stds"] = stds

    temp_order = pickle_save_and_open(temp_order, "temp_order")
